In [3]:
"""
Feature : diabetes
Extract from diagnoses_icd using ICD codes
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [5]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load diagnoses
diagnoses = pd.read_csv(
    os.path.join(hosp_path, "diagnoses_icd.csv"),
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"]
)
diagnoses["icd_code"] = diagnoses["icd_code"].astype(str)

# Function to check if any ICD code matches patterns
def has_icd(icd_list, patterns):
    return icd_list.str.startswith(tuple(patterns)).any()

# Extract diabetes feature
diabetes_df = (
    diagnoses
    .groupby(["subject_id", "hadm_id"])
    .apply(
        lambda g: pd.Series({
            "diabetes": has_icd(g["icd_code"], ["250", "249", "E08", "E09", "E10", "E11", "E12", "E13"])
        }),
        include_groups=False  
    )
    .reset_index()
)

# Merge with dataset
df = df.merge(
    diabetes_df,
    on=["subject_id", "hadm_id"],
    how="left"
)

# Fill missing values with False
df["diabetes"] = df["diabetes"].fillna(False).astype(bool)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nFeature 'diabetes' added.")
print(f"Value counts:\n{df['diabetes'].value_counts()}")
print(f"Total rows: {len(df)}")
print(f"True count: {df['diabetes'].sum()}")
print(f"False count: {(~df['diabetes']).sum()}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")

C:\Users\Coditas\AppData\Local\Temp\ipykernel_3660\2413819149.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["diabetes"] = df["diabetes"].fillna(False).astype(bool)



Feature 'diabetes' added.
Value counts:
diabetes
False    111188
True      46832
Name: count, dtype: int64
Total rows: 158020
True count: 46832
False count: 111188
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 41)


In [6]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [7]:
# cols_to_drop = [
#  "diabetes"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)